In [3]:
# Own Libraries
from utils.data import load_data
from utils.metrics import prec_recall, iou_score, f1_dice
from utils.similarity import Similarity
from utils.image_processing import image_to_windows, get_3d_norm_histogram, calculate_histograms
# 3rd Party Libraries
from skimage.io import imread
from skimage.color import rgb2gray
from typing import Tuple, List
from tqdm import tqdm
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import pickle
import ml_metrics as metrics

In [2]:
pca = PCA().fit(db_feature_matrix)
plt.rcParams["figure.figsize"] = (20,10)
fig, ax = plt.subplots()
xi = np.arange(1, 288, step=1)
y = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0,1.1)
plt.plot(xi, y, marker='o', linestyle='--', color='b')

plt.xlabel('Number of Components')
plt.xticks(np.arange(0, 288, step=5)) #change from 0-based array index to 1-based human-readable label
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 0.85, '95% cut-off threshold', color = 'red', fontsize=16)

ax.grid(axis='x')
plt.show()

NameError: name 'db_feature_matrix' is not defined

In [4]:
"""
   Retrieves the top k similar images for a vector.
"""    
def get_top_k_vector(similarity_vector: np.ndarray, k: int) -> List[str]:
   # We get top K index of the vector (unordered)
   idx = np.argpartition(similarity_vector, -k)[-k:]
   
   # Then we order index in order to get the ordered top k values
   top_k = list(similarity_vector[idx])
   sorted_top = list(sorted(top_k,reverse=True))
   idx = [idx[top_k.index(i)] for i in sorted_top]
   
   # ImageCollection also saves in .files so we can easily retrieve them
   return [db_files[i] for i in idx]


"""
   Retrieves the top k similar images for a QuerySEt
"""    
def get_top_k(similarity_matrix: np.ndarray, k: int, desc: str) -> List[List[str]]:
   # Estimate top k values for all the Queryet
   return [get_top_k_vector(vector,k) for vector in tqdm(similarity_matrix, desc = desc)]


In [12]:
"""
   Plot 1st Query Results
"""   
def plot_image_and_similar(qs,top_qs) -> None:
    
    for i in range(len(qs)):
        f, axarr = plt.subplots(1,2, figsize=(10,10))
        base = qs[i]
        print(top_qs[i])
        query = imread(top_qs[i][0])
        axarr[0].imshow(base)
        axarr[0].title.set_text("Base")
        axarr[1].imshow(query)
        axarr[1].title.set_text("Query")
        plt.show()


"""
    Image path to ID
"""
def get_image_id(image: str) -> str:
    # Extract BBBD_XXX.jpg from relative path
    file = image.split("/")[3]
    # Extract XXX id from BBBD_XXX.jpg 
    id = file.replace(".jpg","").split("_")[1]
    return int(id)


"""
    Save Results properly formated
"""
def save_results(results: List[List[str]], path: str, save: bool) -> None:
    # Vectorize function to apply to each element in results
    get_ids = np.vectorize(get_image_id)
    results = get_ids(results)

    
    # Creates Save Folder 
    if not os.path.exists(path):
        os.makedirs(path)

    if save:
    # Saves data
        pickle.dump(obj = results,file = open(path+"/result.pkl","wb"))
        print("Results Saved!")
    return results



In [ ]:
results = [i for i in range(len(qst2_w2_files))]
for element in idx:
    topk = []
    topk.append(top_k_qst2_w2[element[0][1]])
    if len(element) == 2:
        topk.append(top_k_qst2_w2[element[1][1]])
    
    results[element[0][0]] = topk

result = []
for element in results:   
    tmp = []
    for image in element:
        a = [get_image_id(string) for string in image]
        tmp.append(a)

    if len(tmp) == 1:
        tmp = tmp[0]
    
    result.append(tmp)

pickle.dump(obj = result,file = open("./week2/QST2/method1"+"/result.pkl","wb"))

    

In [6]:
data = {'data/qst2_w2/00000.jpg': [(566, 92, 1243, 639), (74, 178, 296, 458)],
 'data/qst2_w2/00001.jpg': (64, 114, 1879, 2196),
 'data/qst2_w2/00002.jpg': (92, 98, 740, 941),
 'data/qst2_w2/00003.jpg': (94, 85, 1809, 2037),
 'data/qst2_w2/00004.jpg': (109, 0, 1613, 2276),
 'data/qst2_w2/00005.jpg': [(2231, 60, 3960, 1509), (137, 89, 1838, 1541)],
 'data/qst2_w2/00006.jpg': (85, 87, 557, 664),
 'data/qst2_w2/00007.jpg': [(74, 82, 635, 636), (172, 227, 574, 582)],
 'data/qst2_w2/00008.jpg': [(506, 74, 780, 428), (100, 95, 340, 397)],
 'data/qst2_w2/00009.jpg': (0, 82, 3602, 2424),
 'data/qst2_w2/00010.jpg': (0, 0, 3613, 1883),
 'data/qst2_w2/00011.jpg': (58, 85, 678, 681),
 'data/qst2_w2/00012.jpg': (70, 0, 472, 542),
 'data/qst2_w2/00013.jpg': (63, 125, 581, 643),
 'data/qst2_w2/00014.jpg': (61, 0, 664, 788),
 'data/qst2_w2/00015.jpg': (0, 0, 2150, 859),
 'data/qst2_w2/00016.jpg': (80, 62, 1715, 2022),
 'data/qst2_w2/00017.jpg': [(61, 58, 461, 610), (135, 153, 391, 502)],
 'data/qst2_w2/00018.jpg': [(0, 129, 1245, 882), (93, 156, 360, 742)],
 'data/qst2_w2/00019.jpg': (82, 81, 517, 643),
 'data/qst2_w2/00020.jpg': (115, 0, 1501, 1263),
 'data/qst2_w2/00021.jpg': [(647, 139, 986, 435), (68, 180, 496, 378)],
 'data/qst2_w2/00022.jpg': (133, 134, 1945, 1623),
 'data/qst2_w2/00023.jpg': (83, 134, 332, 407),
 'data/qst2_w2/00024.jpg': (84, 101, 475, 612),
 'data/qst2_w2/00025.jpg': (68, 112, 519, 509),
 'data/qst2_w2/00026.jpg': (150, 91, 1236, 1445),
 'data/qst2_w2/00027.jpg': (131, 143, 589, 709),
 'data/qst2_w2/00028.jpg': [(744, 141, 1415, 628), (100, 166, 558, 535)],
 'data/qst2_w2/00029.jpg': [(107, 129, 592, 541), (0, 0, 159, 634)]}

In [7]:
def crop_qst2():
    cropped = []
    cropped2 = []
    for key in data:
        idx = list(qst2_w2_files).index("./"+key)
        image = qst2_w2[idx]

        qs = []

        s = data[key]
        if len(s) == 4:
            s = [s]
        for points in s:
            c = image[points[1]:points[3],points[0]:points[2]]
            qs.append((idx,len(cropped2)))
            cropped2.append(c)

        cropped.append(qs)

    return cropped, cropped2

In [14]:
# Load Data
db, db_files = load_data("./data/BBDD/",".jpg", desc = "Loading BBDD Data...")
qsd2_w1, qsd2_w1_files = load_data("./data/qsd2_w1/",".jpg", desc = "Loading qsd2_w1 Data...")
qsd1_w2, qsd1_w2_files = load_data("./data/qsd1_w2/",".jpg", desc = "Loading qsd1_w2 Data...")
qsd2_w2, qsd2_w2_files = load_data("./data/qsd2_w2/",".jpg", desc = "Loading qsd2_w2 Data...")
qst1_w2, qst2_w2_files = load_data("./data/qst1_w2/",".jpg", desc = "Loading qst1_w2 Data...")
qst2_w2, qst2_w2_files = load_data("./data/qst2_w2/",".jpg", desc = "Loading qst2_w2 Data...")

new_qsd2_w1, masks_1 = crop_images(qsd2_w1,method="otsu")
idx, new_qst2_w2 = crop_qst2()
#new_qsd1_w2, masks_1 = crop_images(qsd1_w2,method="otsu")
#new_qsd2_w2, masks_1 = crop_images(qsd2_w2,method="otsu")


# 3D Normalized Histograms Multiresolutio/Block Images
db_feature_matrix = calculate_histograms(db,16,n_rows=4,n_cols=4,desc="Normalized 3D Histograms Calculation for BBDD...")
qsd2_w1_feature_matrix = calculate_histograms(new_qsd2_w1,16,n_rows=4,n_cols=4,desc = "Normalized 3D Histograms Calculation for qsd2_w1...")
qsd1_w2_feature_matrix = calculate_histograms(qsd1_w2,16,n_rows=4,n_cols=4,desc = "Normalized 3D Histograms Calculation for qsd1_w2...")
qsd2_w2_feature_matrix = calculate_histograms(qsd2_w2,16,n_rows=4,n_cols=4,desc = "Normalized 3D Histograms Calculation for qsd2_w2...")
qst1_w2_feature_matrix = calculate_histograms(qst1_w2,16,n_rows=4,n_cols=4,desc = "Normalized 3D Histograms Calculation for qst1_w2...")
qst2_w2_feature_matrix = calculate_histograms(new_qst2_w2,16,n_rows=4,n_cols=4,desc = "Normalized 3D Histograms Calculation for qst2_w2...")


pca = PCA(n_components=0.95)
db_feature_matrix = pca.fit_transform(db_feature_matrix)
qsd2_w1_feature_matrix = pca.transform(qsd2_w1_feature_matrix)
qsd1_w2_feature_matrix = pca.transform(qsd1_w2_feature_matrix)
qsd2_w2_feature_matrix = pca.transform(qsd2_w2_feature_matrix)
qst1_w2_feature_matrix = pca.transform(qst1_w2_feature_matrix)
qst2_w2_feature_matrix = pca.transform(qst2_w2_feature_matrix)

# Similarity
sim = Similarity()
qs2_w1_similarities = sim.compute_similarities(qs = qsd2_w1_feature_matrix, db_feature_matrix = db_feature_matrix, desc = "Computing qsd2_w1 similarities...", similarity = 'hellinger')
qs1_w2_similarities = sim.compute_similarities(qs = qsd1_w2_feature_matrix, db_feature_matrix = db_feature_matrix, desc = "Computing qsd1_w2 similarities...", similarity = 'hellinger')
qs2_w2_similarities = sim.compute_similarities(qs = qsd2_w2_feature_matrix, db_feature_matrix = db_feature_matrix, desc = "Computing qsd2_w2 similarities...", similarity = 'hellinger')
qst1_w2_similarities = sim.compute_similarities(qs = qst1_w2_feature_matrix, db_feature_matrix = db_feature_matrix, desc = "Computing qst1_w2 similarities...", similarity = 'hellinger')
qst2_w2_similarities = sim.compute_similarities(qs = qst2_w2_feature_matrix, db_feature_matrix = db_feature_matrix, desc = "Computing qst2_w2 similarities...", similarity = 'hellinger')

# Get top K
top_k_qsd2_w1 = get_top_k(similarity_matrix=qs2_w1_similarities,k=10,desc="Retrieving qsd2_w1 top K similar images...")
top_k_qsd1_w2 = get_top_k(similarity_matrix=qs1_w2_similarities,k=10,desc="Retrieving qsd1_w2 top K similar images...")
top_k_qsd2_w2 = get_top_k(similarity_matrix=qs2_w2_similarities,k=10,desc="Retrieving qsd2_w2 top K similar images...")
top_k_qst1_w2 = get_top_k(similarity_matrix=qst1_w2_similarities,k=10,desc="Retrieving qst1_w2 top K similar images...")
top_k_qst2_w2 = get_top_k(similarity_matrix=qst2_w2_similarities,k=10,desc="Retrieving qst2_w2 top K similar images...")

predicted_results = save_results(top_k_qst1_w2,"./week2/QST1/method1",save=True)


# Evaluation and Saving
predicted_results = save_results(top_k_qsd2_w1,"./week1/QST2/method1",save=False)
expected_results = pickle.load(open('./data/qsd2_w1/gt_corresps.pkl', "rb"))
metric = metrics.mapk(actual=expected_results,predicted=predicted_results,k=1)
print("MAP@K Score(ostsu): {:.4f}% ({}/{})".format(metric*100,int(len(predicted_results)*metric),len(predicted_results)))

Loading qsd2_w1 Data...:  23%|██▎       | 7/30 [00:00<00:00, 69.04it/s]

./data/BBDD/ read: 287 images


Loading qsd1_w2 Data...:  17%|█▋        | 5/30 [00:00<00:00, 40.34it/s]

./data/qsd2_w1/ read: 30 images


Loading qsd2_w2 Data...:   3%|▎         | 1/30 [00:00<00:02,  9.73it/s]

...

./data/qst1_w2/ read: 30 images


Cropping Images using Otsu method...:  13%|█▎        | 4/30 [00:00<00:00, 28.24it/s]

./data/qst2_w2/ read: 30 images


Normalized 3D Histograms Calculation for BBDD...: 100%|██████████| 287/287 [00:01<00:00, 189.77it/s]
Normalized 3D Histograms Calculation for qsd2_w1...: 100%|██████████| 30/30 [00:00<00:00, 406.70it/s]
Normalized 3D Histograms Calculation for qsd1_w2...: 100%|██████████| 30/30 [00:00<00:00, 434.71it/s]
Normalized 3D Histograms Calculation for qsd2_w2...: 100%|██████████| 30/30 [00:00<00:00, 107.91it/s]
Normalized 3D Histograms Calculation for qst1_w2...: 100%|██████████| 30/30 [00:00<00:00, 537.75it/s]
Normalized 3D Histograms Calculation for qst2_w2...: 100%|██████████| 39/39 [00:00<00:00, 338.51it/s]
Retrieving qst2_w2 top K similar images...: 100%|██████████| 39/39 [00:00<00:00, 40680.89it/s]

Results Saved!
MAP@K Score(ostsu): 3.3333% (1/30)


In [8]:
# White Background Black Letters
def get_cropped_contours(im,kernel_size=(10,7)):
    lower = np.array([0, 0, 0])
    upper = np.array([35, 35, 35])
    mask = cv2.inRange(im, lower, upper)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
    dilate = cv2.dilate(mask, kernel, iterations=2)

    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]


    candidates = []

    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        area = h*w
        if area >= 20000 and h >= 200:
            text = im[y:y + h, x:x + w]
            candidates.append((area, text))
    
    return candidates

def crop(im):
    
    candidates = get_cropped_contours(im)

    try:
        cropped = [max(candidates)[1]]
        
    except:
        i_1,j_1 = 3,1
        i_2,j_2 = 2,2
        candidates = get_cropped_contours(im,(7,7))
        while not candidates:
            i_1,j_1 = i_1+2,j_1+1
            i_2,j_2 = i_1+2,j_1+2

            candidates = get_cropped_contours(im,(i_1,j_1))
            if not candidates:
                candidates = get_cropped_contours(im,(i_2,j_2))

    
    try:
        cropped = [max(candidates)[1]]    
        candidates.remove(max(candidates))
    except:
        print(candidates)
        
    if len(candidates) >= 3:
        cropped.append(max(candidates)[1])
    
    f, axarr = plt.subplots(1,len(cropped)+1, figsize=(10,10))

    axarr[0].imshow(im)
    for i in range(len(cropped)):
        axarr[i+1].imshow(cropped[i])

    plt.show()
    return cropped



In [9]:
def otsus_binarization(gray_image: np.ndarray) -> float:

    # Histogram
    hist, bin_edges = np.histogram(gray_image, bins=256)

    # Medium from bins
    bin_mids = (bin_edges[:-1] + bin_edges[1:]) / 2.

    # Get the probabilities w1(t), w2(t) with the accumulative probability/distribution of intensities
    w1 = np.cumsum(hist)
    w2 = np.cumsum(hist[::-1])[::-1]

    # Class 1 Mean
    sigma1 = np.cumsum(hist * bin_mids) / w1
    
    # Class 2 Mean
    sigma2 = (np.cumsum((hist * bin_mids)[::-1]) / w2[::-1])[::-1]

    # Variances
    variance_between_classes = w1[:-1] * w2[1:] * (sigma1[:-1] - sigma2[1:]) ** 2

    # Maximize the inter_class_variance function val aka the threshold
    idx_max_variance = np.argmax(variance_between_classes)
    th = bin_mids[:-1][idx_max_variance]
    
    # print("Threshold found: {} ({} if not normalized)".format(th,th*255))
    
    # Generate Mask
    gray_image[gray_image >= th] = 1
    gray_image[gray_image <= th] = 0
    
    
    return gray_image


def crop_image(mask: np.ndarray) -> List[List[int]]:
    mid_i, mid_j = int(mask.shape[0]/2),int(mask.shape[1]/2)
    offset = 50
    
    mid_row = mask[mid_i,:]
    mid_col = mask[:,mid_j]
    
    left = np.where(mid_row == 0)[0][0]
    right = np.where(mid_row == 0)[0][-1]
    top = np.where(mid_col == 0)[0][0]
    bot = np.where(mid_col == 0)[0][-1]
    
    new_mask = np.zeros_like(mask)
    new_mask[top:bot,left:right] = 1
    
    return new_mask, (top,bot,left,right)


def crop_image_2(mask: np.ndarray) -> List[List[int]]:
    # Mid Column and Mid Row
    mid_i, mid_j = int(mask.shape[0]/2),int(mask.shape[1]/2)
    mid_row = mask[mid_i,:]
    mid_col = mask[:,mid_j]
    # Offset to initial points
    offset = 50
    th = 0.15
    
    top = None
    bot = None
    left = None
    right = None
    
    # Search left boundary
    found = False
    i = offset
    

    while not found:
        if abs(mid_row[i] - mid_row[i-1]) >= th: 
            found = True
            left = i
        i += 1


    # Search right boundary
    found = False
    i = mid_row.shape[0] - 1
    while not found:
        if abs(mid_row[i] - mid_row[i-1]) >= th:
            found = True
            right = i
        i -= 1


    # Search top boundary
    found = False
    j = 0
    while not found:
        if abs(mid_col[j] - mid_col[j-1]) >= th:
            found = True
            top = j
        j += 1

    # Search bot boundary
    found = False
    j = mid_col.shape[0] - 1
    while not found:
        if abs(mid_col[j] - mid_col[j-1]) >= th:
            found = True
            bot = j
        j -= 1

    new_mask = np.zeros_like(mask)
    new_mask[top:bot,left:right] = 1

    return new_mask, (top,bot,left,right)



def crop_images(qs: np.ndarray, method: str) -> Tuple[List[np.ndarray], List[np.ndarray]]:
    masks, cropped_images = [], []
    
    if method == "otsu":
        for image in tqdm(qs, desc= "Cropping Images using Otsu method..."):
            im = otsus_binarization(rgb2gray(image))
            mask, coords = crop_image(im)
            cropped_image = image[coords[0]:coords[1], coords[2]:coords[3],:]
            masks.append(mask)
            cropped_images.append(cropped_image)
                    
    elif method == "diff":
        for image in tqdm(qs, desc= "Cropping Images using Pixel Diff method..."):
            im = rgb2gray(image)
            try:
                mask, coords = crop_image_2(im)
                cropped_image = image[coords[0]:coords[1], coords[2]:coords[3]]
            except:
                mask = np.zeros_like(image)
                mask[int(mask.shape[0]*0.25):,:,:] = [1,1,1]
                mask[int(mask.shape[0]*0.75):,:] = [1,1,1]
                mask[:,:int(mask.shape[1]*0.25),:] = [1,1,1]
                mask[:,int(mask.shape[1]*0.75):,:] = [1,1,1]
                cropped_image = image[int(image.shape[0]*0.25):int(image.shape[0]*0.75),int(image.shape[1]*0.25):int(image.shape[1]*0.75),:]
            
            masks.append(mask)
            cropped_images.append(cropped_image)
            

                
    return cropped_images, masks

In [ ]:
def plot_image_and_windows(images: np.ndarray, n_cols: int, n_rows: int) -> None:
    for im in images:
        windows = image_to_windows(im, n_cols=n_cols, n_rows=n_rows)
        f, axarr = plt.subplots(4,4, figsize=(15,15))
        for i in range(n_rows):
            for j in range(n_cols):
                axarr[i,j].imshow(windows[j+i*n_rows])
                
        plt.savefig('tmp.png')
        plt.clf() 
        f, axarr = plt.subplots(1,2, figsize=(15,15))
        windows = imread('tmp.png')
        axarr[0].imshow(windows)
        axarr[1].imshow(im)
        plt.show()
        os.remove('tmp.png')
        
        
plot_image_and_windows(db[:2], 4, 4)